# 02 - 单变量分析

## 🎯 学习目标
- 掌握数值型变量的分布分析方法
- 掌握类别型变量的频数分析
- 学会使用多种可视化方法展示单变量特征

## 📚 核心知识点
1. **数值型变量分析**
   - 直方图（Histogram）：观察分布形态
   - 箱线图（Boxplot）：识别异常值
   - 密度图（KDE）：平滑的分布曲线
   - QQ图：检验正态性

2. **类别型变量分析**
   - 频数统计
   - 柱状图/饼图
   - 高基数类别特征处理

In [ ]:
# 导入库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# 配置
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False
sns.set_palette('husl')

print("✅ 环境准备完成")

---
## 1. 加载数据

In [ ]:
train = pd.read_pickle('../../data/processed/train_initial.pkl')
print(f"数据形状: {train.shape}")

# 分离特征类型
numeric_features = train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = train.select_dtypes(include=['object']).columns.tolist()

# 假设目标变量
TARGET_COL = 'target'  # 根据实际修改
if TARGET_COL in numeric_features:
    numeric_features.remove(TARGET_COL)

---
## 2. 数值型变量分析

### 💡 学习要点
- **偏度（Skewness）**：分布的对称性
  - 偏度 = 0：正态分布
  - 偏度 > 0：右偏（长尾在右）
  - 偏度 < 0：左偏（长尾在左）
  
- **峰度（Kurtosis）**：分布的陡峭程度
  - 峰度 = 3：正态分布
  - 峰度 > 3：尖峰
  - 峰度 < 3：平峰

In [ ]:
def analyze_numeric_feature(df, feature):
    """分析单个数值型特征"""
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f'特征: {feature}', fontsize=16, fontweight='bold')
    
    # 1. 直方图 + KDE
    axes[0, 0].hist(df[feature].dropna(), bins=50, edgecolor='black', alpha=0.7)
    axes[0, 0].set_title('直方图')
    axes[0, 0].set_xlabel(feature)
    axes[0, 0].set_ylabel('频数')
    
    # 2. 密度图
    df[feature].dropna().plot(kind='kde', ax=axes[0, 1], linewidth=2)
    axes[0, 1].set_title('核密度估计 (KDE)')
    axes[0, 1].set_xlabel(feature)
    
    # 3. 箱线图
    axes[1, 0].boxplot(df[feature].dropna(), vert=False)
    axes[1, 0].set_title('箱线图（检测异常值）')
    axes[1, 0].set_xlabel(feature)
    
    # 4. QQ图（正态性检验）
    stats.probplot(df[feature].dropna(), dist="norm", plot=axes[1, 1])
    axes[1, 1].set_title('QQ图（正态性检验）')
    
    plt.tight_layout()
    plt.show()
    
    # 统计信息
    print(f"\n{'='*50}")
    print(f"特征: {feature}")
    print(f"{'='*50}")
    print(f"缺失值: {df[feature].isnull().sum()} ({df[feature].isnull().sum()/len(df)*100:.2f}%)")
    print(f"唯一值数量: {df[feature].nunique()}")
    print(f"\n描述性统计:")
    print(df[feature].describe())
    print(f"\n偏度 (Skewness): {df[feature].skew():.4f}")
    print(f"峰度 (Kurtosis): {df[feature].kurtosis():.4f}")
    
    # 异常值统计（IQR方法）
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[feature] < lower_bound) | (df[feature] > upper_bound)]
    print(f"\n异常值数量 (IQR方法): {len(outliers)} ({len(outliers)/len(df)*100:.2f}%)")

In [ ]:
# 示例：分析第一个数值型特征
if len(numeric_features) > 0:
    analyze_numeric_feature(train, numeric_features[0])

### 批量分析所有数值型特征

In [ ]:
# 创建统计摘要表
numeric_summary = pd.DataFrame({
    '特征': numeric_features,
    '缺失率': [train[col].isnull().sum() / len(train) * 100 for col in numeric_features],
    '唯一值': [train[col].nunique() for col in numeric_features],
    '均值': [train[col].mean() for col in numeric_features],
    '标准差': [train[col].std() for col in numeric_features],
    '最小值': [train[col].min() for col in numeric_features],
    '最大值': [train[col].max() for col in numeric_features],
    '偏度': [train[col].skew() for col in numeric_features],
    '峰度': [train[col].kurtosis() for col in numeric_features]
})

numeric_summary.round(3)

### 可视化所有数值型特征分布

In [ ]:
# 绘制所有数值型特征的直方图
n_cols = 3
n_rows = (len(numeric_features) + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, n_rows * 4))
axes = axes.flatten()

for idx, feature in enumerate(numeric_features):
    train[feature].hist(bins=50, ax=axes[idx], edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'{feature}\n偏度: {train[feature].skew():.2f}')
    axes[idx].set_xlabel('')

# 隐藏多余的子图
for idx in range(len(numeric_features), len(axes)):
    axes[idx].axis('off')

plt.suptitle('所有数值型特征分布', fontsize=16, y=1.001)
plt.tight_layout()
plt.show()

---
## 3. 类别型变量分析

### 💡 学习要点
- 高基数类别特征：唯一值很多的类别特征
- 类别不平衡：某些类别样本量远大于其他类别
- 稀有类别：出现次数很少的类别

In [ ]:
def analyze_categorical_feature(df, feature, top_n=10):
    """分析单个类别型特征"""
    fig, axes = plt.subplots(1, 2, figsize=(16, 5))
    fig.suptitle(f'特征: {feature}', fontsize=16, fontweight='bold')
    
    # 频数统计
    value_counts = df[feature].value_counts()
    
    # 1. 柱状图（显示Top N）
    value_counts.head(top_n).plot(kind='barh', ax=axes[0])
    axes[0].set_title(f'Top {top_n} 类别频数')
    axes[0].set_xlabel('频数')
    
    # 2. 占比图
    value_pct = df[feature].value_counts(normalize=True) * 100
    value_pct.head(top_n).plot(kind='barh', ax=axes[1])
    axes[1].set_title(f'Top {top_n} 类别占比')
    axes[1].set_xlabel('占比 (%)')
    
    plt.tight_layout()
    plt.show()
    
    # 统计信息
    print(f"\n{'='*50}")
    print(f"特征: {feature}")
    print(f"{'='*50}")
    print(f"缺失值: {df[feature].isnull().sum()} ({df[feature].isnull().sum()/len(df)*100:.2f}%)")
    print(f"唯一值数量: {df[feature].nunique()}")
    print(f"\nTop 10 类别:")
    top_10 = pd.DataFrame({
        '类别': value_counts.head(10).index,
        '频数': value_counts.head(10).values,
        '占比(%)': value_pct.head(10).values
    })
    print(top_10.to_string(index=False))
    
    # 稀有类别统计
    rare_threshold = 0.01  # 出现频率小于1%
    rare_categories = value_pct[value_pct < rare_threshold * 100]
    print(f"\n稀有类别（<1%）: {len(rare_categories)} 个")

In [ ]:
# 示例：分析第一个类别型特征
if len(categorical_features) > 0:
    analyze_categorical_feature(train, categorical_features[0])

### 批量分析所有类别型特征

In [ ]:
# 创建类别型特征摘要表
categorical_summary = pd.DataFrame({
    '特征': categorical_features,
    '缺失率': [train[col].isnull().sum() / len(train) * 100 for col in categorical_features],
    '唯一值': [train[col].nunique() for col in categorical_features],
    '最常见类别': [train[col].mode()[0] if len(train[col].mode()) > 0 else None for col in categorical_features],
    '最常见类别占比': [train[col].value_counts(normalize=True).iloc[0] * 100 if len(train[col].value_counts()) > 0 else 0 for col in categorical_features]
})

categorical_summary.round(2)

---
## 4. 识别需要特殊处理的特征

### 💡 关键判断标准
1. **高偏度特征**（|skewness| > 1）：考虑对数变换
2. **高缺失率特征**（>50%）：考虑删除或特殊处理
3. **常数特征**：唯一值=1，应删除
4. **高基数类别特征**：考虑频数编码或目标编码

In [ ]:
print("特征诊断报告")
print("="*60)

# 1. 高偏度特征
high_skew_features = numeric_summary[numeric_summary['偏度'].abs() > 1]['特征'].tolist()
print(f"\n1. 高偏度特征 (|skew| > 1): {len(high_skew_features)} 个")
if high_skew_features:
    print(f"   {high_skew_features}")
    print("   建议: 考虑对数变换或Box-Cox变换")

# 2. 高缺失率特征
high_missing_numeric = numeric_summary[numeric_summary['缺失率'] > 50]['特征'].tolist()
high_missing_categorical = categorical_summary[categorical_summary['缺失率'] > 50]['特征'].tolist()
high_missing_features = high_missing_numeric + high_missing_categorical
print(f"\n2. 高缺失率特征 (>50%): {len(high_missing_features)} 个")
if high_missing_features:
    print(f"   {high_missing_features}")
    print("   建议: 考虑删除或将缺失作为新类别")

# 3. 常数特征
constant_features = [col for col in train.columns if train[col].nunique() == 1]
print(f"\n3. 常数特征: {len(constant_features)} 个")
if constant_features:
    print(f"   {constant_features}")
    print("   建议: 删除，无信息量")

# 4. 高基数类别特征
high_cardinality_features = categorical_summary[categorical_summary['唯一值'] > 50]['特征'].tolist()
print(f"\n4. 高基数类别特征 (唯一值>50): {len(high_cardinality_features)} 个")
if high_cardinality_features:
    print(f"   {high_cardinality_features}")
    print("   建议: 频数编码、目标编码或分组")

---
## 📝 本节总结

### 已完成的工作
- [ ] 数值型特征的分布分析
- [ ] 类别型特征的频数分析
- [ ] 识别异常值和偏态分布
- [ ] 识别需要特殊处理的特征

### 关键发现
记录你的发现：
1. 哪些特征呈正态分布？
2. 哪些特征有严重的偏态？
3. 哪些类别特征不平衡？
4. 有哪些异常值需要处理？

### 下一步计划
- 双变量分析：探索特征与目标的关系
- 多变量分析：特征间的相关性